# GeoPandas Tutorial

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

import init
import whaletracks.common. constants as cn
import obspy

ModuleNotFoundError: No module named 'future'

In [ ]:
import sys
sys.path

## Constants

In [ ]:
GEOMETRY = "geometry"
LON = "LON"
LAT = "LAT"

## USA Data

In [ ]:
# Get county level data
path = os.path.join(cn.PROJECT_DIR, "examples/USAData/USA_adm2.shp")
if not (os.path.isfile(path)):
    print("Invalid path: %s" % path)
gdf_county = gpd.read_file(path)
gdf_county

In [ ]:
gdf_county.plot(figsize=(10, 12))
plt.xlim([-130, -65])
plt.ylim([25,50])

## IRIS Stations

In [ ]:
path = os.path.join(cn.DATA_DIR, "ALL-StationList.csv")
gdf_sta = pd.read_csv(path)
gdf_sta = gdf_sta[gdf_sta[LON] < 0]  # Eliminate outlier
gdf_sta["geometry"] = gpd.GeoSeries([Point(float(x),float(y)) for x, y in zip(gdf_sta[LAT], gdf_sta[LON])])
gdf_sta.head()

In [ ]:
gdf_sta["point"] = gpd.GeoSeries([Point(float(x),float(y)) for x, y in zip(gdf_sta['LAT'], gdf_sta['LON'])])

In [ ]:
plt.scatter(gdf_sta[LON], gdf_sta[LAT])

## Combine Points With Map

In [ ]:
gdf = gpd.GeoDataFrame(pd.concat([gdf_county, gdf_ca[GEOMETRY]]))
gdf = gpd.GeoDataFrame(pd.concat([gdf_ca[GEOMETRY]]))
gdf.columns = [GEOMETRY]
gdf.plot()
#plt.ylim([-120, -100])
#plt.xlim([40, 50])

In [ ]:
gdf_ca

## Select Stations Near the Continental US West Coast

In [ ]:
gdf_west = gdf_county[gdf_county["NAME_1"].isin(["California", "Oregon", "Washington"])]
len(gdf_west)

In [ ]:
def circleDistance(point1, point2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees).
    :param Point point1:
    :param Point point2:
    :return float: distance in miles
    """
    lon1 = point1.y
    lat1 = point1.x
    lon2 = point2.y
    lat2 = point2.x
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # Radius of earth in miles. Use 6371 for kilometers
    return c * r

def getCount(idx):
    ser = gdf_west.iloc[idx]
    return ser[GEOMETRY]
poly1 = getCount(1)
poly3 = getCount(3)
circleDistance(poly1.centroid, poly3.centroid)

In [ ]:
poly1.centroid.y, poly1.centroid.x

In [ ]:
poly3.centroid.y, poly3.centroid.x

In [ ]:
len(gdf_sta)

## ObsPy

In [ ]:
dir(obspy)